In [2]:
import os, math
import numpy as np
import pandas as pd
import pyodbc 

In [3]:
# Specify Database Details
server = '********'
database = 'PT_Bkup'

# Specify the driver, server, and database (trusted connection uses MS Authentication)
cnxn_str = ('DRIVER={ODBC Driver 17 for SQL Server}; \
            SERVER=' + server + '; \
            DATABASE=' + database + '; \
            Trusted_Connection=yes;')

# Import Farm & Roll Adjustment Data from SQL into Pandas dataframes
cnxn = pyodbc.connect(cnxn_str)

raw_data = '''select munid_cur, munupid_cur, sort_municipal_cur, munshort_cur+' '+muntitleabrv_cur 'munshort',  tier_cur, regionshort,
 sum(1) 'Farm Property Count',
 sum(case when roll15 = '511901000304400' then sitearea*.0000229568411 
  when acres is null and measureunit = 'A' then sitearea
  when acres is null and measureunit = 'F' then sitearea*.0000229568411 
  else acres end) 'Farm Acres'
from YuCh.FAM_2023_OMPF a
  inner join v_munsearch b on a.munid = b.xmunid and b.taxyear = '2021' and typeid in ('LT-OWN','ST-OWN','UT-LT')
group by munid_cur, sort_municipal_cur, munshort_cur, muntitleabrv_cur, tier_cur, regionshort, munupid_cur
order by sort_municipal_cur
'''

roll15_data = '''select p.Munid, xRoll15, p.Acres, roll15, ctymn, prevroll15, prevctymn, '2022' 'Year'
from [dbo].[Totals2022] t 
left join [DongJi].[Parcel_xRoll15_Area_GIS] p on t.roll15 = p.xRoll15 
where prevroll15 <> '' and left(roll15,4) <> left(prevroll15,4) and totalassmt > 0 and year = '2022' and version = 'Feb'
group by xRoll15, roll15, ctymn, prevroll15, prevctymn, p.Munid, Acres
order by roll15'''

farms = pd.read_sql(raw_data, cnxn)
roll = pd.read_sql(roll15_data, cnxn)

cnxn.close()


In [4]:
farms.head()

,munid_cur,munupid_cur,sort_municipal_cur,munshort,tier_cur,regionshort,Farm Property Count,Farm Acres
0,1950,1950,1,Toronto C,ST,GTA,5,1244.164413
1,1800,1800,2,Durham R,UT,GTA,4640,321419.433168
2,1813,1800,3,Oshawa C,LT,GTA,159,9611.600032
3,1801,1800,4,Pickering C,LT,GTA,465,23563.919002
4,1805,1800,5,Ajax T,LT,GTA,36,1701.008313


In [5]:
roll.head()

,Munid,xRoll15,Acres,roll15,ctymn,prevroll15,prevctymn,Year
0,3218,321801102036601,28.203848,321801102036601,3218,321101102036601,3211,2022
1,3218,321801102036700,9.953652,321801102036700,3218,321101102036700,3211,2022
2,3218,321801102037050,4.944349,321801102037050,3218,321101102037000,3211,2022
3,3218,321801102044900,77.997375,321801102044900,3218,321101102044900,3211,2022
4,3218,321801102044901,1.097401,321801102044901,3218,321101102044901,3211,2022


In [6]:
# Find the Sum of the adjustments needed for the farm properties in the roll dataframe
Adjustment = roll['Acres'].agg(np.sum)

# Lookup the LT & UT Municipality Name for the adjusted farm properties 
plus = farms["munid_cur"][farms["munid_cur"]==roll["Munid"][0]].index[0]
minus = farms["munid_cur"][farms["munid_cur"]==roll["prevctymn"][0]].index[0]

# Print out the name of the Municipality 
print(farms["munshort"][plus],roll["Munid"][0], plus)
print(farms["munshort"][minus],roll["prevctymn"][0], minus)

Ingersoll T 3218 63
South-West Oxford Tp 3211 68


In [45]:
# Import Statistics Canada Municipal Area Acres 2021 Census Data 
statscan_df = pd.read_excel(r'Population and Municipal Land Area - 2021 Census.xls',sheet_name='munlist',header=2, skiprows=3)

# Rename the last two columns in the dataframe to reflect the UT Roll Up 
statscan_df.rename(columns={'Population, 2021.1': 'Population, 2021 (UT Roll Up)', 'Land area in acres, 2021.1': 'Land area in acres, 2021 (UT Roll Up)'}, inplace=True)

In [46]:
# Convert Farms Munid Column to int to enable the merging of datasets
farms["munid_cur"] = farms["munid_cur"].astype(int)

# Merge the StatsCan Data set with the Farms dataset 
data = pd.merge(left=statscan_df,right=farms,how='left',on='munid_cur')

# View the merged dataframe 
data

,munid_cur,munupid_cur_x,tier_cur_x,regionShort,munname_cur,sort_municipal_cur_x,xsc_id,muntier,"Population, 2021","Land area in square kilometres, 2021","Land area in acres, 2021","Population, 2021 (UT Roll Up)","Land area in acres, 2021 (UT Roll Up)",munupid_cur_y,sort_municipal_cur_y,munshort,tier_cur_y,regionshort,Farm Property Count,Farm Acres
0,1950,1950,ST,GTA,Toronto C,1,3520005,1950ST,2794356.0,631.10,155947.9655,2794356,155947.96550,1950,1.0,Toronto C,ST,GTA,5.0,1244.164413
1,1800,1800,UT,GTA,Durham R,2,3518,1800UT,NaN,NaN,NaN,696867,622341.35565,1800,2.0,Durham R,UT,GTA,4640.0,321419.433168
2,1813,1800,LT,GTA,Oshawa C,3,3518013,1800LT,175383.0,145.72,36008.1406,175383,36008.14060,1800,3.0,Oshawa C,LT,GTA,159.0,9611.600032
3,1801,1800,LT,GTA,Pickering C,4,3518001,1800LT,99186.0,231.10,57105.9655,99186,57105.96550,1800,4.0,Pickering C,LT,GTA,465.0,23563.919002
4,1805,1800,LT,GTA,Ajax T,5,3518005,1800LT,126666.0,66.64,16467.0772,126666,16467.07720,1800,5.0,Ajax T,LT,GTA,36.0,1701.008313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,5462,5400,ST,Northeast,Larder Lake Tp,451,3554062,5400ST,745.0,227.62,56246.0401,745,56246.04010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
439,5458,5400,ST,Northeast,McGarry Tp,452,3554058,5400ST,579.0,85.62,21157.1301,579,21157.13010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440,5466,5400,ST,Northeast,Gauthier Tp,453,3554066,5400ST,151.0,87.98,21740.2979,151,21740.29790,NaN,NaN,NaN,NaN,NaN,NaN,NaN
441,5456,5400,ST,Northeast,Matachewan Tp,454,3554056,5400ST,268.0,539.56,133327.9738,268,133327.97380,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# Drop redundant columns from the dataframe 
data.drop(['munupid_cur_x','sort_municipal_cur_x','xsc_id',
            'sort_municipal_cur_y','munupid_cur_y','muntier','munshort',
            'tier_cur_y','regionshort','Land area in square kilometres, 2021',
            'Land area in acres, 2021','Population, 2021'], axis=1, inplace=True)

In [48]:
# Find the index of the two adjustment municipalities in the data dataframe 
print(farms["munshort"][plus],roll["Munid"][0])
print(farms["munshort"][minus],roll["prevctymn"][0])

Ingersoll T 3218
South-West Oxford Tp 3211


In [49]:
# Save the indices into varaibles to be used in the loop 
add = data["munname_cur"][data["munname_cur"]==farms["munshort"][plus]]
subtract = data["munname_cur"][data["munname_cur"]==farms["munshort"][minus]]

In [50]:
# View Pre-Adjusted values 
pre_adjusted_add_value = data["Land area in acres, 2021 (UT Roll Up)"][add.index[0]]
pre_adjusted_sub_value = data["Land area in acres, 2021 (UT Roll Up)"][subtract.index[0]]
print(pre_adjusted_add_value,pre_adjusted_sub_value)

# Replace the two values that need to be adjusted with their adjusted values 
data["Land area in acres, 2021 (UT Roll Up)"][add.index[0]] = data["Land area in acres, 2021 (UT Roll Up)"][add.index[0]] + Adjustment
data["Land area in acres, 2021 (UT Roll Up)"][subtract.index[0]] = data["Land area in acres, 2021 (UT Roll Up)"][subtract.index[0]] - Adjustment

# View Post-Adjusted Values
post_adjusted_add_value = data["Land area in acres, 2021 (UT Roll Up)"][add.index[0]]
post_adjusted_sub_value = data["Land area in acres, 2021 (UT Roll Up)"][subtract.index[0]]
print(post_adjusted_add_value,post_adjusted_sub_value)

3145.64665 91332.47905
3932.95348586506 90545.17221413493


<ipython-input-50-d4753618c2e0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Land area in acres, 2021 (UT Roll Up)"][add.index[0]] = data["Land area in acres, 2021 (UT Roll Up)"][add.index[0]] + Adjustment
<ipython-input-50-d4753618c2e0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Land area in acres, 2021 (UT Roll Up)"][subtract.index[0]] = data["Land area in acres, 2021 (UT Roll Up)"][subtract.index[0]] - Adjustment


In [51]:
# Divide the Municipalities Farm Acres by the Total Land to get the 2023 FAM
data["2023 FAM"] = data['Farm Acres'] / data["Land area in acres, 2021 (UT Roll Up)"]

# Round to the 3rd decimal place
data["2023 FAM"] = data["2023 FAM"].round(decimals=4)

In [52]:
# View descriptive statistics for 2023 FAM 
data["2023 FAM"].describe()

count    392.000000
mean       0.380417
std        0.300521
min        0.000200
25%        0.085575
50%        0.347250
75%        0.636775
max        0.928100
Name: 2023 FAM, dtype: float64

In [32]:
# View the Adjusted Added Record 
data.loc[data["munname_cur"]==farms["munshort"][plus]]

,munid_cur,tier_cur_x,regionShort,munname_cur,"Population, 2021 (UT Roll Up)","Land area in acres, 2021 (UT Roll Up)",Farm Property Count,Farm Acres,2023 FAM
64,3218,LT,Southwest,Ingersoll T,13693,3932.953486,19.0,831.085176,0.211


In [33]:
# View Adjusted Subtracted Records 
data.loc[data["munname_cur"]==farms["munshort"][minus]]

,munid_cur,tier_cur_x,regionShort,munname_cur,"Population, 2021 (UT Roll Up)","Land area in acres, 2021 (UT Roll Up)",Farm Property Count,Farm Acres,2023 FAM
69,3211,LT,Southwest,South-West Oxford Tp,7583,90545.172214,904.0,79440.201454,0.877


In [61]:
# Replace NaNs with 0 for the whole dataframe 
data = data.fillna(0)

In [62]:
# Check Georgian Bay which has a zero for 2023 FAM and a number of other columns
data.loc[data["munname_cur"]=='Georgian Bay Tp']

,munid_cur,tier_cur_x,regionShort,munname_cur,"Population, 2021 (UT Roll Up)","Land area in acres, 2021 (UT Roll Up)",Farm Property Count,Farm Acres,2023 FAM
59,4465,LT,Central,Georgian Bay Tp,3441,129801.78545,0.0,0.0,0.0


In [64]:
# Save the Dataframe to the working directory
workdir = os.getcwd()
data.to_csv(workdir + '\\2023_FAM_Recalculation.csv')